In [4]:
pwd

u'/Users/auroraleport/Documents/LePort_git/SQL_practice'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [42]:
sales_df = pd.read_excel('SQL_practice.xlsx', 'SALESPERSON')

In [43]:
orders_df = pd.read_excel('SQL_practice.xlsx', 'ORDERS')

In [44]:
customer_df = pd.read_excel('SQL_practice.xlsx', 'CUSTOMER')

In [45]:
sales_df.head()

,ID,Name,Age,Salary
0,1,Abe,61,140000
1,2,Bob,34,44000
2,5,Chris,34,40000
3,7,Dan,41,52000
4,8,Ken,57,115000


In [46]:
orders_df.head()

,Number,order_date,cust_id,salesperson_id,Amount
0,10,1996-08-02,4,2,540
1,20,1999-01-30,4,8,1800
2,30,1995-07-14,9,1,460
3,40,1998-01-29,7,2,2400
4,50,1998-02-03,6,7,600


In [47]:
customer_df.head()

,ID,Name,City,Industry Type
0,4,Samsonic,pleasant,J
1,6,Panasung,oaktown,J
2,7,Samony,jackson,B
3,9,Orange,Jackson,B


# Connecting SQL 

In [48]:
dfs = [sales_df, customer_df, orders_df]
names = ['sales', 'customer', 'orders']

In [49]:
orders_df.columns

Index([u'Number', u'order_date', u'cust_id', u'salesperson_id', u'Amount'], dtype='object')

In [29]:
import sqlite3 as sql
import sys 

In [30]:
#conn = sql.connect('SQLcapstone.db')
#conn.text_factory = str
#c = conn.cursor()

In [50]:
for name,df in zip(names,dfs):
    with sql.connect('sales_info.db') as conn:
        conn.text_factory = str
        df.to_sql(name, conn, flavor='sqlite',if_exists='replace',index=False)
c = conn.cursor()        

In [51]:
c.execute("""CREATE TABLE or_cust_saleID as 
          SELECT customer.ID, customer.Name, orders.cust_id, orders.salesperson_id
          FROM customer
          JOIN orders
          WHERE customer.ID = orders.cust_id""")

In [60]:
c.execute("""SELECT * FROM or_cust_saleID""" )
names = [description[0] for description in c.description]
print names
data = c.fetchall()
table1 = pd.DataFrame(data, columns = ['ID', 'Name', 'cust_id', 'salesperson_id'])

['ID', 'Name', 'cust_id', 'salesperson_id']


In [91]:
table1

,ID,Name,cust_id,salesperson_id
0,4,Samsonic,4,2
1,4,Samsonic,4,8
2,6,Panasung,6,7
3,6,Panasung,6,7
4,7,Samony,7,2
5,9,Orange,9,1
6,9,Orange,9,7


In [92]:
c.execute("""DROP TABLE joined_table""")

In [93]:
c.execute("""CREATE TABLE joined_table AS
              SELECT or_cust_saleID.ID, or_cust_saleID.name, or_cust_saleID.cust_id, \
              or_cust_saleID.salesperson_id, sales.name as sales_name, sales.salary
              FROM or_cust_saleID
              JOIN sales
              WHERE sales.ID = or_cust_saleID.salesperson_id """)

In [95]:
c.execute("""SELECT * FROM joined_table""")
names = [description[0] for description in c.description]
print names
data = c.fetchall()
table2 = pd.DataFrame(data, columns = ['ID', 'Name', 'cust_id', 'salesperson_id', 'sales_name', 'salary'])
table2

['ID', 'Name', 'cust_id', 'salesperson_id', 'sales_name', 'Salary']


,ID,Name,cust_id,salesperson_id,sales_name,salary
0,4,Samsonic,4,2,Bob,44000
1,4,Samsonic,4,8,Ken,115000
2,6,Panasung,6,7,Dan,52000
3,6,Panasung,6,7,Dan,52000
4,7,Samony,7,2,Bob,44000
5,9,Orange,9,1,Abe,140000
6,9,Orange,9,7,Dan,52000


In [96]:
c.execute("""SELECT sales_name, Name
            FROM joined_table
            WHERE Name = 'Samsonic'""")
data = c.fetchall()
pd.DataFrame(data)

,0,1
0,Bob,Samsonic
1,Ken,Samsonic


In [119]:
c.execute("""SELECT sales_name, Name
            FROM joined_table
            WHERE Name != 'Samsonic'""")
data = c.fetchall()
pd.DataFrame(data)

,0,1
0,Dan,Panasung
1,Dan,Panasung
2,Bob,Samony
3,Abe,Orange
4,Dan,Orange


In [98]:
c.execute("""SELECT count(sales_name), sales_name
             FROM joined_table
             GROUP BY sales_name
             HAVING count(sales_name) >= 2""")
data = c.fetchall()
pd.DataFrame(data)

,0,1
0,2,Bob
1,3,Dan


In [114]:
c.execute("""DROP TABLE highAchiever""")

In [115]:
c.execute("""CREATE TABLE highAchiever AS
            SELECT sales_name, salary
            FROM joined_table
            WHERE salary >= 100000""")

In [117]:
c.execute("""SELECT * FROM highAchiever""")
data = c.fetchall()
pd.DataFrame(data)

,0,1
0,Ken,115000
1,Abe,140000


In [ ]:
c.execute()